In [1]:
import csv, json
from gql import gql, Client
from gql.transport.requests import RequestsHTTPTransport
from copy import deepcopy
import pandas as pd

In [2]:
def cross_join(left, right):
    new_rows = [] if right else left
    for left_row in left:
        for right_row in right:
            temp_row = deepcopy(left_row)
            for key, value in right_row.items():
                temp_row[key] = value
            new_rows.append(deepcopy(temp_row))
    return new_rows


def flatten_list(data):
    for elem in data:
        if isinstance(elem, list):
            yield from flatten_list(elem)
        else:
            yield elem


def json_to_dataframe(data_in):
    def flatten_json(data, prev_heading=''):
        if isinstance(data, dict):
            rows = [{}]
            for key, value in data.items():
                rows = cross_join(rows, flatten_json(value, prev_heading + '.' + key))
        elif isinstance(data, list):
            rows = []
            for i in range(len(data)):
                [rows.append(elem) for elem in flatten_list(flatten_json(data[i], prev_heading))]
        else:
            rows = [{prev_heading[1:]: data}]
        return rows

    return pd.DataFrame(flatten_json(data_in))


In [3]:
url='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v2'

sample_transport=RequestsHTTPTransport(
   url=url,
   verify=True,
   retries=3,
)
client = Client(
   transport=sample_transport
)


In [4]:
triplets = pd.read_excel("v3_triplets.xlsx")
tokenid = pd.read_excel("v3_tokenid.xlsx")
tokenid_dict = dict(zip(tokenid.token, tokenid.id))

### V2 only
###get id of pair 
query_pairid = gql('''

query ($tok0: String!, $tok1: String!){
      pairs (where :{token0 : $tok0, token1 :$tok1}){
      id
    }
}

''')
params = {
    "tok0": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    "tok1": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"
    
}
response = client.execute(query_pairid, variable_values=params)
pairs_list = []

#need to test tok0 tok1 pair in both directions
for i,row in triplets.iterrows():
    pair_params11 =  {
    "tok0": tokenid_dict[row.token1.rstrip()],
    "tok1": tokenid_dict[row.token2.rstrip()]
    }
    pair_params12 =  {
    "tok1": tokenid_dict[row.token1.rstrip()],
    "tok0": tokenid_dict[row.token2.rstrip()]
    }
    pair_params21 =  {
    "tok0": tokenid_dict[row.token2.rstrip()],
    "tok1": tokenid_dict[row.token3.rstrip()]
    }
    pair_params22 =  {
    "tok1": tokenid_dict[row.token2.rstrip()],
    "tok0": tokenid_dict[row.token3.rstrip()]
    }
    pair_params31 =  {
    "tok0": tokenid_dict[row.token1.rstrip()],
    "tok1": tokenid_dict[row.token3.rstrip()]
    }
    pair_params32 =  {
    "tok1": tokenid_dict[row.token1.rstrip()],
    "tok0": tokenid_dict[row.token3.rstrip()]
    }
    response11 = client.execute(query_pairid, variable_values=pair_params11)
    response12 = client.execute(query_pairid, variable_values=pair_params12)
    response21 = client.execute(query_pairid, variable_values=pair_params21)
    response22 = client.execute(query_pairid, variable_values=pair_params22)
    response31 = client.execute(query_pairid, variable_values=pair_params31)
    response32 = client.execute(query_pairid, variable_values=pair_params32)
    if len(response11['pairs']) != 0:
        pairs_list.append(response11['pairs'][0]['id'])
    if len(response12['pairs']) != 0:
        pairs_list.append(response12['pairs'][0]['id'])
    if len(response21['pairs']) != 0:
        pairs_list.append(response21['pairs'][0]['id'])
    if len(response22['pairs']) != 0:
        pairs_list.append(response22['pairs'][0]['id'])
    if len(response31['pairs']) != 0:
        pairs_list.append(response31['pairs'][0]['id'])
    if len(response32['pairs']) != 0:
        pairs_list.append(response32['pairs'][0]['id'])
          
pairs_list= list(set(pairs_list))

In [24]:
### V3 only
### query given pairs
urlv3='https://api.thegraph.com/subgraphs/name/uniswap/uniswap-v3'

sample_transportv3=RequestsHTTPTransport(
   url=urlv3,
   verify=True,
   retries=3,
)
clientv3 = Client(
   transport=sample_transportv3
)

In [26]:
query_pairid = gql('''

query ($tok0: String!, $tok1: String!){
      pools (where :{token0 : $tok0, token1 :$tok1}){
      id
    }
}

''')
params = {
    "tok0": "0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2",
    "tok1": "0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48"
    
}
response = clientv3.execute(query_pairid, variable_values=params)


In [31]:
pairs_list = []
#need to test tok0 tok1 pair in both directions
for i,row in triplets.iterrows():
    pair_params11 =  {
    "tok0": tokenid_dict[row.token1.rstrip()],
    "tok1": tokenid_dict[row.token2.rstrip()]
    }
    pair_params12 =  {
    "tok1": tokenid_dict[row.token1.rstrip()],
    "tok0": tokenid_dict[row.token2.rstrip()]
    }
    pair_params21 =  {
    "tok0": tokenid_dict[row.token2.rstrip()],
    "tok1": tokenid_dict[row.token3.rstrip()]
    }
    pair_params22 =  {
    "tok1": tokenid_dict[row.token2.rstrip()],
    "tok0": tokenid_dict[row.token3.rstrip()]
    }
    pair_params31 =  {
    "tok0": tokenid_dict[row.token1.rstrip()],
    "tok1": tokenid_dict[row.token3.rstrip()]
    }
    pair_params32 =  {
    "tok1": tokenid_dict[row.token1.rstrip()],
    "tok0": tokenid_dict[row.token3.rstrip()]
    }
    response11 = clientv3.execute(query_pairid, variable_values=pair_params11)
    response12 = clientv3.execute(query_pairid, variable_values=pair_params12)
    response21 = clientv3.execute(query_pairid, variable_values=pair_params21)
    response22 = clientv3.execute(query_pairid, variable_values=pair_params22)
    response31 = clientv3.execute(query_pairid, variable_values=pair_params31)
    response32 = clientv3.execute(query_pairid, variable_values=pair_params32)
    if len(response11['pools']) != 0:
        pairs_list.append(response11['pools'][0]['id'])
    if len(response12['pools']) != 0:
        pairs_list.append(response12['pools'][0]['id'])
    if len(response21['pools']) != 0:
        pairs_list.append(response21['pools'][0]['id'])
    if len(response22['pools']) != 0:
        pairs_list.append(response22['pools'][0]['id'])
    if len(response31['pools']) != 0:
        pairs_list.append(response31['pools'][0]['id'])
    if len(response32['pools']) != 0:
        pairs_list.append(response32['pools'][0]['id'])
          
pairs_list= list(set(pairs_list))

In [79]:
query_pairvolume = gql('''
query ($pairid: String!) {poolDayDatas(first: 1000, where: {pool: $pairid}){
      date
      id
      volumeUSD
    pool {
      feeTier
      id
      token0{
        id
        symbol
          }
      token1{
        id
        symbol
          }
        }
    }
}
''')

combined_df = pd.DataFrame()

for pairid in pairs_list:
    params = {"pairid": pairid}
    response = clientv3.execute(query_pairvolume, variable_values=params)
    temp_df = json_to_dataframe(response)
    if len(temp_df)>=100:
        combined_df = pd.concat([combined_df, temp_df], ignore_index=True)
        

In [81]:
combined_df.to_csv("pairs_volume_V3_11032022.csv")

In [67]:
pairs_list

['0x84c35ec407a8d1caae6dac975a30cf1492df2c18',
 '0x091c0158ab410bd73ca1541409d5a22e90146a04',
 '0x60594a405d53811d3bc4766596efd80fd545a270',
 '0x3470447f3cecffac709d3e783a307790b0208d60',
 '0x3139bbba7f4b9125595cb4ebeefdac1fce7ab5f1',
 '0x5859ebe6fd3bbc6bd646b73a5dbb09a5d7b6e7b7',
 '0x3afdc5e6dfc0b0a507a8e023c9dce2cafc310316',
 '0x4c83a7f819a5c37d64b4c5a2f8238ea082fa1f4e',
 '0x69d91b94f0aaf8e8a2586909fa77a5c2c89818d5',
 '0xab2044f105c43c25b1de3ee27504f0b889ce5953',
 '0xa15cc73e881c06d8db06b50b7a3688b763c18350',
 '0x4768f4dfb46b26889ffaab881945bd7d340505c2',
 '0x68f73e2180024db5b54e0e119d4f5128953f9417',
 '0x3019d4e366576a88d28b623afaf3ecb9ec9d9580',
 '0xd83d78108dd0d1dffff11ea3f99871671a52488b',
 '0x82743c07bf3be4d55876f87bca6cce5f84429bd0',
 '0xb06e7ed37cfa8f0f2888355dd1913e45412798c5',
 '0x81216f193e8bed640a2378c38d689ebacc4b5d2c',
 '0x24ee2c6b9597f035088cda8575e9d5e15a84b9df',
 '0x7210e71c96e002ecc784f4be6f0cb5ce35e49b2d',
 '0x5864dea5f1750d1f8887f9fb7f3a50f15789514e',
 '0xde77450d0